In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from FoodSeg103_data_handler import *

## Create scribbles

Load the ground truths as batch

In [ ]:
img_nums = [n for n in range(0, 4900, 8)] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

Check and filter for resolution

In [ ]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 250_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

In [ ]:
# Filter out those images
gts = {im_num: gt for im_num, gt in gts.items() if resol[im_num] <= thresh}
num_imgs = len(gts)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Define scribbles parameters

In [ ]:
# Which scribbles to create
bins = [0.05, 0.3, 1.5]
suff = ["w3"] #standing for width 3
sq_scaling = 100
scribble_width = 3
scribbles_seed = 1

# Where to save the scribbles
folder_path = "./FoodSeg103_results/data/run02"

save_res = True
show_res = False
print_steps = True

Loop and create scribbles

In [ ]:
percentages = {}
for bin in bins:
    percentages[bin] = []
    for img_num, gt in gts.items():
        for s in suff:
            np.random.seed(scribbles_seed)
            print(f"IMG {img_num}: bin {bin}, suff {s}")
            scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, sq_scaling=sq_scaling, mode="all",
                                                            save_res=save_res, suff=s, show_res=show_res, image=None, print_steps=print_steps, scribble_width=scribble_width)
            percentages[bin].append(perc_labelled)
            print(f"Annotation percentage: {perc_labelled:2.2f}%\n")

Quick report of labelling percentages:

In [ ]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

Show the last created scribble with its ground truth, and all ground truths (optionally)

In [ ]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")

In [ ]:
for g in gts:
    v.add_labels(gts[g], name=f"GT {g}")